In [2]:
from keras.models import load_model
import cv2
import numpy as np
from keras.preprocessing import image
import operator
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


Using TensorFlow backend.


In [3]:
model = load_model(' overfitting_adding_weight_regularization_L2_1.h5')

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('ADELE.jpg')
img = cv2.resize(img,(150,150))
img = np.reshape(img,[1,150,150,3])

classes = model.predict_classes(img)

print(classes)

W0718 12:29:40.534971 140603958323008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0718 12:29:40.553807 140603958323008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0718 12:29:40.589272 140603958323008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0718 12:29:40.892453 140603958323008 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0718 12:29:40.893119 140603958323008 deprecation_

[[1]]


In [4]:

# dimensions of our images
img_width, img_height = 150, 150

In [5]:
# load the model we saved
model = load_model(' overfitting_adding_weight_regularization_L2_1.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:327: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [6]:
def getKeysByValue(dictOfElements, valueToFind):
    listOfKeys = list()
    listOfItems = dictOfElements.items()
    for item  in listOfItems:
        if item[1] == valueToFind:
            listOfKeys.append(item[0])
    return  listOfKeys

In [10]:
predict_image_path = 'ADELE.jpg'

img = cv2.imread(predict_image_path)
#img = image.load_img(predict_image_path, target_size=(150, 150))
img = np.random.randint(0,10,(150,150,3))
img.shape
img = np.expand_dims(img, axis=0)
img.shape

predict = model.predict(img)
predicted_class_indices = {'tanker': 0, 'yacht': 1}
print(predicted_class_indices)
print(predict)

index, value = max(enumerate(predict), key=operator.itemgetter(1))
print(max(value))

max_predict = predict.argmax(axis=1)[0]
print(max_predict)



print(getKeysByValue(predicted_class_indices, max_predict)[0])

{'tanker': 0, 'yacht': 1}
[[0.9019618]]
0.9019618
0
tanker


In [9]:
import cv2

orig = cv2.imread(predict_image_path)
cv2.putText(orig, "Predicted: {}({:.2f})%".format(getKeysByValue(predicted_class_indices, max_predict)[0], max(value) *100), (10, 30),
                cv2.FONT_HERSHEY_PLAIN, 1.5, (43, 99, 255), 2)

cv2.imshow("Classification", orig)
cv2.waitKey(10000)
cv2.destroyAllWindows()

In [20]:
base_dir = '/home/jose/Desktop/ship_dataset_6500_1000_625_yacht_tanker/'
pic_size = 150
batch_size = 20


datagen_validation = ImageDataGenerator(rescale=1./255)

validation_generator = datagen_validation.flow_from_directory(base_dir + "validation",
                                                               target_size=(150, 150),
                                                               batch_size=20,
                                                               class_mode='binary')

Found 2000 images belonging to 2 classes.


In [21]:
#filenames = validation_generator.filenames
#nb_samples = len(filenames)
#predictions = model.predict_generator(generator=validation_generator, steps = nb_samples)

filenames = validation_generator.filenames
nb_samples = len(filenames)

predictions = model.predict_generator(validation_generator, steps = nb_samples)
y_pred = [np.argmax(probas) for probas in predictions]
y_test = validation_generator.classes
class_names = validation_generator.class_indices.keys()
print(class_names)


dict_keys(['tanker', 'yacht'])


In [22]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes, title='Confusion matrix', cmap=plt.cm.Blues):
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    
# compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, title='Normalized confusion matrix')
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [2000, 40000]